## 모델 구성에 사용되는 패키지(모듈들) 불러오기

> 전제조건

- Python 실행환경이 구성됨 (assets의 requirements.txt를 참고해주세요.)

- 실험용 데이터 셋이 준비됨 (dataset_creation.ipynb를 참고해주세요.)

In [1]:
import os
import sys

from IPython.display import clear_output

__FILE_DIR_PATH = os.path.dirname(__vsc_ipynb_file__) \
    if __IPYTHON__ \
    else os.path.dirname(__file__)
    
WORKSPACE_HOME = __FILE_DIR_PATH.replace(
    f"/ipynb/{os.path.basename(__FILE_DIR_PATH)}", "")
sys.path.append(WORKSPACE_HOME)
DATASET_DIR_HOME = f"{WORKSPACE_HOME}/data/colley"

from core import *
from colley import *
from ipirec import *

데이터 불러오기

In [2]:
# 첫번째 교차 검증 데이터 집합을 선택합니다.
KFOLD_NO = 0

# 데이터 환경 및 메타데이터를 불러옵니다.
_dataset = ColleyDataSetRev(DATASET_DIR_HOME)

# (KFOLD_NO + 1)번째 의사결정 데이터를 불러옵니다.
_dataset.load_kfold_train_set(KFOLD_NO)

[IO] /home/taegyu/repo/ipirec/data/colley/train_0_view_list.csv


[LOAD] train_0_view_list.csv: 100%|██████████| 83481/83481 [00:01<00:00, 70666.92it/s]


[IO] /home/taegyu/repo/ipirec/data/colley/train_0_like_list.csv


[LOAD] train_0_like_list.csv: 100%|██████████| 144804/144804 [00:02<00:00, 58246.05it/s]


[IO] /home/taegyu/repo/ipirec/data/colley/train_0_purchase_list.csv


[LOAD] train_0_purchase_list.csv: 100%|██████████| 1876/1876 [00:00<00:00, 40786.20it/s]


[INFO]
- Users    :      5976
- Items    :       429
- Tags     :       258
- view     :     13854
- like     :     18854
- purchase :      1434
[INFO]
- Users:      5976
- Items:       429
- Tags :       258



모델 만들기 (IPIRec Rev4)

In [3]:
_model = IPIRecModelSeries3(_dataset)
_model.analysis()

IPIRecModelSeries3.preprocess()


tags_score: 100%|██████████| 258/258 [00:00<00:00, 1679.33it/s]


예측기 만들기 (예측, 예측에 대한 피드백 기능 추가하기)
- 훈련관련 변수 정하기 (학습율, 일반화 등)
    - 실험으로 찾은 값을 기본으로 하도록 구성했습니다.
    - 형식적으로 함수가 반환한 dict를 사용하시면 됩니다.
- 예측기 객체 생성하기

In [4]:
_estimator_params = IPIRecEstimatorSeries3.create_models_parameters()
_estimator = IPIRecEstimatorSeries3(
    _model, 
    _estimator_params,
    )

IPIRecEstimatorSeries3.append_biases()


훈련 조건으로 의사결정 종류별 순서가 주어져야합니다.
- TRAIN_DTYPE_SEQ
    - 주 처리 단계는 봤다, 좋다, 샀다 순으로 이뤄집니다(기본 값).
- POST_TRAIN_DTYPE_SEQ
    - 후 처리 단계는 이뤄지지 않지만(기본 값), 필요하다면 열거자들을 추가하면 됩니다.

#### 여기까지가 분석모델 구성입니다.

In [ ]:
# 주 처리 훈련 순서
TRAIN_DTYPE_SEQ = [_ for _ in DecisionType]
# 후 처리 훈련 순서
POST_TRAIN_DTYPE_SEQ = []

_estimator.train(
    TRAIN_DTYPE_SEQ,
    POST_TRAIN_DTYPE_SEQ,
    )

#### 추천하기
훈련된 모델을 사용해서, 모든 항목들에 대한 긍정의 의사결정 정도를 예측합니다.

점수기반 추천목록 생성은 다음을 따릅니다: 

In [47]:
_recommender = ScoreBasedRecommender(
    _estimator,
)

긍정집단기반 추천목록 생성은 다음을 따릅니다. (주석을 제거해 사용)

In [46]:
"""
_recommender = ELABasedRecommender(
    _estimator,
)
"""

사용자들이 의사결정이 없는 모든 항목들의 긍정할 정도를 예측합니다.

In [10]:
_recommender.prediction()

ScoreBasedRecommender.prediction(): 100%|██████████| 5976/5976 [02:00<00:00, 49.57it/s] 


사용자 u의 항목 i에 대한 긍정정도 예측은 다음과 같습니다.

In [67]:
# user_id = 708083
user_id = 764602
item_id = 81
user: UserEntity = _recommender.user_dict[user_id]
item: ItemEntity = _recommender.item_dict[item_id]

_estimated_score = _recommender.predict(user_id, item_id)
print(f"[User] {user_id}\n- Interest tags: {user.set_of_interest_tags}")
print(f"[Item] {item_id}\n- {item.item_name}")
print(_estimated_score)

[User] 764602
- Interest tags: {'짱구는못말려'}
[Item] 81
- 월레스와 그로밋 숀 발매트
0.5221187954424878


해당 사용자의 구매내역 (없으면 미출력)

In [70]:
_decision_type = DecisionType.E_PURCHASE
_test_set_file_path = _dataset.kfold_file_path(KFOLD_NO, _decision_type, False)
_top_n_conditions = [n for n in range(3, 19, 2)]
_evaluator = IRMetricsEvaluator(_recommender, _test_set_file_path)
_evaluator.top_n_eval(_top_n_conditions)
_user: UserEntity = _evaluator.user_dict.get(user_id, None)
if _user == None:
    exit()
_actual_items = _user.dict_of_decision_item_ids[DecisionType.to_str(_decision_type)]
for _item_id in _actual_items:
    _item: ItemEntity = _recommender.item_dict.get(_item_id, None)
    if _item == None:
        continue
    print(f"{_item.item_id:5d}\t {_item.item_name:40s}\t {_item.tags_set}")

[IO] /home/taegyu/repo/ipirec/data/colley/test_0_purchase_list.csv


[LOAD] test_0_purchase_list.csv: 100%|██████████| 469/469 [00:00<00:00, 38815.46it/s]


추천내역

In [71]:
_user: UserEntity = _recommender.user_dict[user_id]
if _user == None:
    exit()
for _item_id, _score in _user.recommended_items_dict.items():
    _item: ItemEntity = _recommender.item_dict.get(_item_id, None)
    if _item == None:
        continue
    print(f"{_item.item_id:5d}\t {_item.item_name:40s}\t {_item.tags_set}")

   20	 원피스 쵸파 구름 블루투스 스피커                      	 {'원피스', '무드등'}
   50	 해리포터 폴리주스 포션 무드등 S                      	 {'무드등', '인테리어', '해리포터', '조명'}
   61	 곽철이 시리얼볼 1P                             	 {'접시', '그릇', '곽철이'}
   80	 월레스와 그로밋 머그컵 - 모자 그로밋                   	 {'월레스와그로밋', '인테리어', '장난감', '머그컵'}
   82	 곽철이 물티슈 케이스                             	 {'케이스', '곽철이'}
   91	 짱구는못말려 대형 짱구피규어                         	 {'피규어', '짱구는못말려'}
   92	 곽철이 뭉뚝 머그&소서 세트                         	 {'그릇', '곽철이', '슬리퍼', '접시', '인테리어', '머그컵'}
   94	 빤쮸토끼 스마트 베드 테이블 2종                      	 {'인테리어', '빤쮸토끼', '테이블'}
   96	 곽철이 겨울 털슬리퍼 2종                          	 {'곽철이', '슬리퍼'}
   97	 곽철이 크리스마스 패브릭 포스터                       	 {'포스터', '곽철이'}
  101	 월레스와 그로밋 애프터눈 티 접시 3P 세트                	 {'월레스와그로밋', '그릇', '접시', '테이블'}
34867	 ???                                     	 {'호빵맨', '피규어', '장난감', '해피밀'}
36020	 ??빈티지 세일러문??                            	 {'키링', '피규어', '세일러문'}
36146	 ?카오하나 가라케?                              	 {'헬로

성능평가

In [72]:
_evaluator.top_n_eval(_top_n_conditions)
print(_evaluator.evlautions_summary_df())

   Conditions  Precision    Recall  F1-score  Accuracy  Hits   TP    FP   FN  \
0           3   0.043758  0.112957  0.063080  0.990910    34   34   743  267   
1           5   0.045560  0.196013  0.073935  0.986698    59   59  1236  242   
2           7   0.046884  0.282392  0.080416  0.982504    85   85  1728  216   
3           9   0.050622  0.392027  0.089666  0.978436   118  118  2213  183   
4          11   0.061074  0.578073  0.110476  0.974782   174  174  2675  127   
5          13   0.059400  0.664452  0.109051  0.970588   200  200  3167  101   
6          15   0.051995  0.671096  0.096512  0.965962   202  202  3683   99   
7          17   0.045878  0.671096  0.085884  0.961300   202  202  4201   99   

       TN  
0  110067  
1  109574  
2  109082  
3  108597  
4  108135  
5  107643  
6  107127  
7  106609  
